# Depocenter migration

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [ ]:
def loadTIN(folder, timestep):
    #
    for i in range(0, 1):
        df = h5py.File('%s/tin.time%s.p%s.hdf5'%(folder, timestep, i), 'r')
        coord = np.array((df['/coords']))
        EroDep = np.array((df['/cumdiff']))
        if i == 0:
            x1, y1, z1 = np.hsplit(coord, 3)
            erodep = EroDep

    pointx = x1
    pointy = y1
    pointz = z1

    dx = x1[1]-x1[0]

    nx = int((pointx.max() - pointx.min())/dx + 1)
    ny = int((pointy.max() - pointy.min())/dx + 1)
    xi = np.linspace(pointx.min(), pointx.max(), nx)
    yi = np.linspace(pointy.min(), pointy.max(), ny)
    # Create the regular mesh
    xi, yi = np.meshgrid(xi, yi)
    erodep = griddata((pointx[:,0],pointy[:,0]),erodep[:,0],(xi,yi),method='nearest')

    return erodep

In [ ]:
nbout = 31
nstep = 10

In [ ]:
folder = '/workspace/volume/GRL/rf3_sl3c_ts1_dtn/h5/'
EroDep_all = {}
for i in range(0,nbout = 31):
    EroDep_all[i] = loadTIN(folder, i*nstep)

## Calculate sedimentation rate

In [ ]:
cumDiff = []
for i in range(nbout):
    for j in range(0,350):
        cumDiff.append(strat_all[i].erodep[350][j])
# 
CumDiff = np.reshape(cumDiff, (nbout, 350))
SedRate = np.zeros((nbout-1,350))
for i in range(nbout-1):
    for j in range(0,350):
        SedRate[i][j] = CumDiff[i+1][j]-CumDiff[i][j]

In [ ]:
# Find the depocenter through time
# Depocenter is defined as the position with maximum deposition rate
depocenter = []
for i in range(nbout-1):
    depocenter.append(SedRate[i].argmax())

In [ ]:
mpl.rc("figure", figsize=(6, 3))
ax = sns.heatmap(SedRate[0:21], vmin=-20, vmax=20, cmap="bwr", xticklabels=50, yticklabels=4, linewidth=0, cbar=False)
# ax.cax.set_visible(False)
ax.set_position([0.12,0.2, 0.81, 0.74])
time=np.linspace(0,30,31)
plt.plot(depocenter[0:21],time[0:21]+0.5,'o', markersize=4, color='yellow')
ax.set_xlim([0,300])
ax.set_ylim([0,20])
plt.ylabel('Time (Myr)',fontsize=12)
plt.xlabel('Distance (km)',fontsize=12)
# 
# ax.figure.savefig("/workspace/volume/GRL/figures/depocenter_sedrate.png", dpi=400)